In [1]:
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['Company','Rating','Salary','Job','Speciality','Skills'])

# Чтобы спарсить все вакансии пока достаточно обойти 56 страниц
for i in range(56):
    html_page = urlopen("https://career.habr.com/vacancies?page="+str(i)+"&type=all&with_salary=true")
    soup = BeautifulSoup(html_page, "html.parser")
    for link in soup.findAll('div', class_='vacancy-card__inner'):
        df.loc[len(df)] = [
            link.find('div', class_='vacancy-card__company-title').text if link.find('div', class_='vacancy-card__company-title') else None,
            link.find('div', class_='vacancy-card__company-rating').text if link.find('div', class_='vacancy-card__company-rating') else None,
            link.find('div', class_='basic-salary').text if link.find('div', class_='basic-salary') else None,
            link.find('div', class_='vacancy-card__title').text if link.find('div', class_='vacancy-card__title') else None,
            link.find('div', class_='vacancy-card__skills').text.split('•')[0] if link.find('div', class_='vacancy-card__skills') else None,
            link.find('div', class_='vacancy-card__skills').text.split('•')[1:]] if link.find('div', class_='vacancy-card__skills') else None

In [2]:
df['Skills'] = [[item.strip() for item in row] for row in df['Skills']]

df['MinSalary'] = \
df['Salary'].str.replace(' ','').str.extract(r'^(от)*([0-9]+)')[1] \
if df['Salary'].str.replace(' ','').str.extract(r'^(от)*([0-9]+)')[1] is not None \
else df['Salary'].str.replace(' ','').str.extract(r'([0-9]+)')[1]

df['MaxSalary'] = \
df['Salary'].str.replace(' ','').str.extract(r'(до)+([0-9]+)')[1] \
if df['Salary'].str.replace(' ','').str.extract(r'(до)+([0-9]+)')[1] is not None \
else df['Salary'].str.replace(' ','').str.extract(r'([0-9]+)')[1]

df['MinSalary'] = df['MinSalary'].mask(pd.isnull, df['MaxSalary'])
df['MaxSalary'] = df['MaxSalary'].mask(pd.isnull, df['MinSalary'])
df['AvgSalary'] = (df['MaxSalary'].astype('float')+df['MinSalary'].astype('float'))/2.
df['Currency'] = df['Salary'].str[-1]
# Необходима единственная валюта для расчета зарплат. Возьмем 1$=100₽ и 1€ = 100₽
df['RubAvgSalary'] = np.where(df['Currency'] == '₽', df['AvgSalary'], df['AvgSalary']*100)
df['SkillsCount'] = df['Skills'].apply(len)
df['SkillSalary'] = df['AvgSalary'] / df['SkillsCount']
df['Level'] = df['Speciality'].str.extract('(Intern|Junior|Middle|Senior|Lead)')

In [3]:
# Полученные данные выглядят следующим образом
df

,Company,Rating,Salary,Job,Speciality,Skills,MinSalary,MaxSalary,AvgSalary,Currency,RubAvgSalary,SkillsCount,SkillSalary,Level
0,Автомакон,None,от 250 000 ₽,Data Engineer,"Инженер по данным, Старший (Senior)","[SQL, Python, Apache Airflow, Greenplum, Apach...",250000,250000,250000.0,₽,250000.0,5,50000.000000,Senior
1,Автомакон,None,от 28 200 ₽,Релиз-менеджер,"Релиз менеджер, Средний (Middle)","[Git, Agile]",28200,28200,28200.0,₽,28200.0,2,14100.000000,Middle
2,Автомакон,None,от 300 000 ₽,Инженер DevOps,"DevOps-инженер, Средний (Middle)","[Администрирование Linux, Docker, Python, Bash]",300000,300000,300000.0,₽,300000.0,4,75000.000000,Middle
3,Автомакон,None,от 80 000 ₽,Аналитик (исследование ИТ-рынка и стратегическ...,"Маркетинговый аналитик, Средний (Middle)","[Microsoft Excel, Анализ данных, Математическа...",80000,80000,80000.0,₽,80000.0,4,20000.000000,Middle
4,Wanted.,​4.71​,до 220 000 ₽,1C Программист,"Программист 1С, Средний (Middle)",[1C: ERP],220000,220000,220000.0,₽,220000.0,1,220000.000000,Middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,ЭКОПСИ Консалтинг,None,от 140 000 до 180 000 ₽,Fullstack\Backend Web-разработчик,"Фулстек разработчик, Средний (Middle)","[C#, JavaScript, HTML, CSS, .NET, ASP.NET MVC]",140000,180000,160000.0,₽,160000.0,6,26666.666667,Middle
550,DevCube Innovations,None,от 2100 до 2600 $,Strong Middle PHP LARAVEL Developer - Remote,"Фулстек разработчик, Средний (Middle)","[PHP, JavaScript, HTML, CSS, Laravel]",2100,2600,2350.0,$,235000.0,5,470.000000,Middle
551,INDEX,None,до 800 000 ₽,SRE/DEVOPS,"Инженер по доступности сервисов, Старший (Seni...","[Linux, Высоконагруженные системы, Git]",800000,800000,800000.0,₽,800000.0,3,266666.666667,Senior
552,М Тех,None,от 87 000 до 96 000 ₽,Старший инженер поддержки (2 линия),"Инженер технической поддержки, Старший (Senior)","[Zabbix, Grafana, ELK Stack]",87000,96000,91500.0,₽,91500.0,3,30500.000000,Senior


In [4]:
# Создаем матрицу скиилов. На пересечении i-ой строки (вакансии) и j-ого столбца (скилла)
# значение равно 1,если текущий скилл представлен в данной вакансии и 0, если нет

skills_array = df.explode('Skills')['Skills'].unique()
skills_array = pd.array(skills_array.tolist()+['{{SYSTEM}} Свободный коэффициент'])

skills_matrix = np.zeros((len(df),len(skills_array)))
skills_matrix.shape

(554, 496)

In [5]:
salary_array = df['RubAvgSalary']
len(salary_array)

554

In [6]:
for i, row in df.iterrows():
    for j, skill in enumerate(skills_array):
        if any(value == skill for value in row['Skills']):
            skills_matrix[i,j] = 1
    skills_matrix[i,-1] = 1

In [7]:
# Решаем уравнение вида A*X = B. Где A - skills_matrix, B - массив зарплат.
# Решением является массив X - ожидаемая оценка зарплаты текущей зарплаты

x = np.linalg.lstsq(skills_matrix, np.array(salary_array), rcond=None)[0]

In [8]:
df_salary = pd.DataFrame(columns=['Skill','ExpSalary'])

for i, value in enumerate(x):
    df_salary.loc[len(df_salary)] = [skills_array[i],value]

In [9]:
# Выводим пары (навык; ожидаемая оценка)
# На текущий момент оценка происходит приоритетным способом.
# Больше приоритет - более высокая корреляция с высокой зарплатой
df_salary

,Skill,ExpSalary
0,SQL,-60880.382318
1,Python,-67182.255975
2,Apache Airflow,-140641.229873
3,Greenplum,114621.017924
4,Apache Spark,106018.536402
...,...,...
491,Разработка креативных стратегий коммуникаций,-5149.352161
492,Стратегические коммуникации,-5149.352161
493,Разработка решений по интеграции,-240749.144217
494,ASP.NET MVC,208315.365512


In [10]:
# Можно сохранить
df_salary.to_csv('csvs/SkillExpSalary.csv')